## Διαβάζουμε το dataset

In [3]:
import pandas as pd

df = pd.read_excel('../data/housing_data_gr.xlsx')

In [4]:
len(df)

20000

## Αρχικά θα διαγράψουμε τα χαρακτηριστικά που δεν θέλουμε

In [5]:
df=df.drop(columns=['deleted_at','deleted',"location_region","res_address"])

### Θα διαχειριστούμε τα προβλήμτατ του dataset π.χ στις θεσεις parking null=false

In [6]:
df['parking'] = df['parking'].notna()
df['parking'] = df['parking'].astype(int)



## Θα φτιάξω μια στήλη για τις τιμές ενοικίων

In [7]:
df["rent_price"]=(df["res_price"]*0.005).astype(int)

### Διαγραφω τα records που περιεχουν null τιμές

In [8]:
df = df.dropna()

### Διαγραφω τα διπλοτυπα

In [9]:
duplicate_rows = df.duplicated().sum()
if duplicate_rows>0:
    df=df.drop_duplicates()

### Κωδικοποιω τα κατηγορικά

In [10]:
level_mapping = {
    "Ισόγειο": 0,
    "1ος": 1,
    "2ος": 2,
    "3ος": 3,
    "4ος": 4,
    "5ος": 5,
    "6ος": 6,
    "7ος": 7,
    "8ος": 8
}
df=df[df["res_type"]!="Κτίριο"]
    
df["levels"] = df["levels"].map(level_mapping)
df['levels'] = df['levels'].fillna(0)

In [11]:
df=pd.get_dummies(df, columns=['res_type'], prefix='res_type', drop_first=True)
cols = [col for col in df.columns if col.startswith('res_type_')]
df[cols] = df[cols].astype(int)


In [12]:
df['res_date'] = pd.to_datetime(df['res_date'], errors='coerce')

df['res_date'] = df['res_date'].dt.year

In [13]:
df = pd.get_dummies(df, columns=['location_name'], prefix='location_name', drop_first=True)

cols = [col for col in df.columns if col.startswith('location_name')]
df[cols] = df[cols].astype(int)

In [14]:
ordinal_mapping = {
    'Α+': 10,
    'Α': 9,
    'Β+': 8,
    'Β': 7,
    'Γ': 6,
    'Δ': 5,
    'Ε': 4,
    'Ζ': 3,
    'Η': 2,
    'Μη αποδοτικό': 1,      
    'Εκρεμμεί': 0,          
    'Εξαιρείται': -1       
}

df['energyclass'] = df['energyclass'].map(ordinal_mapping)

In [15]:
condition_mapping = {
    'Νεόδμητο': 8,             # Top condition (brand new)
    'Άριστη': 7,               # Excellent condition
    'Ανακαινισμένο': 6,        # Recently renovated, very good
    'Καλή': 5,                 # Good, well-maintained
    'Χρήζει ανακαίνισης': 4,   # Needs renovation, but functional
    'Ημιτελές': 3,             # Incomplete, might need significant work to finish
    'Υπο κατασκευή': 2,        # Not yet habitable
    'Άλλη κατάσταση': 1,      # Special case, could mean anything, assigning a low but distinct value
    # You could also consider np.nan for 'Άλλη κατάσταση' if it truly means unknown/unspecifiable.
}

df['status'] = df['status'].map(condition_mapping)

In [16]:
df.to_excel('../data/dataset_ready_for_pre_proc.xlsx', index=False)